In [1]:
import pandas as pd
import numpy as np
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
data = pd.read_csv("Electronics.csv")

In [3]:
data.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,"Redmi 10 Power (Power Black, 8GB RAM, 128GB St...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/81eM15lVcJ...,https://www.amazon.in/Redmi-Power-Black-128GB-...,4.0,965,"₹10,999","₹18,999"
1,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 6GB RAM,...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/71AvQd3Vzq...,https://www.amazon.in/OnePlus-Nord-Lite-128GB-...,4.3,"113,956","₹18,999","₹19,999"
2,OnePlus Bullets Z2 Bluetooth Wireless in Ear E...,"tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/51UhwaQXCp...,https://www.amazon.in/Oneplus-Bluetooth-Wirele...,4.2,"90,304","₹1,999","₹2,299"
3,"Samsung Galaxy M33 5G (Mystique Green, 6GB, 12...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/81I3w4J6yj...,https://www.amazon.in/Samsung-Mystique-Storage...,4.1,"24,863","₹15,999","₹24,999"
4,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 6GB RAM...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/I/71V--WZVUI...,https://www.amazon.in/OnePlus-Nord-Black-128GB...,4.3,"113,956","₹18,999","₹19,999"


In [4]:
data.duplicated().sum()

np.int64(0)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9600 entries, 0 to 9599
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            9600 non-null   object
 1   main_category   9600 non-null   object
 2   sub_category    9600 non-null   object
 3   image           9600 non-null   object
 4   link            9600 non-null   object
 5   ratings         9505 non-null   object
 6   no_of_ratings   9505 non-null   object
 7   discount_price  9116 non-null   object
 8   actual_price    9530 non-null   object
dtypes: object(9)
memory usage: 675.1+ KB


In [6]:
data.isna().sum()

name                0
main_category       0
sub_category        0
image               0
link                0
ratings            95
no_of_ratings      95
discount_price    484
actual_price       70
dtype: int64

In [7]:
data['#keywords'] = data['name'].str.lower()

In [8]:
data['#keywords'] 

0       redmi 10 power (power black, 8gb ram, 128gb st...
1       oneplus nord ce 2 lite 5g (blue tide, 6gb ram,...
2       oneplus bullets z2 bluetooth wireless in ear e...
3       samsung galaxy m33 5g (mystique green, 6gb, 12...
4       oneplus nord ce 2 lite 5g (black dusk, 6gb ram...
                              ...                        
9595    palay® bts backpack for boys kpop bts bangtan ...
9596    moca ipad air 5th generation case 2022 / ipad ...
9597    zebronics zeb-sound bomb 5 tws v5.0 bluetooth ...
9598    linqs® original nxp chip | waterproof nfc tag ...
9599    pragati systems® medium grain 24x36 inch 10 oz...
Name: #keywords, Length: 9600, dtype: object

In [9]:
data.columns

Index(['name', 'main_category', 'sub_category', 'image', 'link', 'ratings',
       'no_of_ratings', 'discount_price', 'actual_price', '#keywords'],
      dtype='object')

In [10]:
data['main_category'].value_counts()

main_category
tv, audio & cameras    9600
Name: count, dtype: int64

In [11]:
data.drop(['main_category','sub_category'],axis=1,inplace=True)

In [12]:
data

,name,image,link,ratings,no_of_ratings,discount_price,actual_price,#keywords
0,"Redmi 10 Power (Power Black, 8GB RAM, 128GB St...",https://m.media-amazon.com/images/I/81eM15lVcJ...,https://www.amazon.in/Redmi-Power-Black-128GB-...,4.0,965,"₹10,999","₹18,999","redmi 10 power (power black, 8gb ram, 128gb st..."
1,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 6GB RAM,...",https://m.media-amazon.com/images/I/71AvQd3Vzq...,https://www.amazon.in/OnePlus-Nord-Lite-128GB-...,4.3,"113,956","₹18,999","₹19,999","oneplus nord ce 2 lite 5g (blue tide, 6gb ram,..."
2,OnePlus Bullets Z2 Bluetooth Wireless in Ear E...,https://m.media-amazon.com/images/I/51UhwaQXCp...,https://www.amazon.in/Oneplus-Bluetooth-Wirele...,4.2,"90,304","₹1,999","₹2,299",oneplus bullets z2 bluetooth wireless in ear e...
3,"Samsung Galaxy M33 5G (Mystique Green, 6GB, 12...",https://m.media-amazon.com/images/I/81I3w4J6yj...,https://www.amazon.in/Samsung-Mystique-Storage...,4.1,"24,863","₹15,999","₹24,999","samsung galaxy m33 5g (mystique green, 6gb, 12..."
4,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 6GB RAM...",https://m.media-amazon.com/images/I/71V--WZVUI...,https://www.amazon.in/OnePlus-Nord-Black-128GB...,4.3,"113,956","₹18,999","₹19,999","oneplus nord ce 2 lite 5g (black dusk, 6gb ram..."
...,...,...,...,...,...,...,...,...
9595,PALAY® BTS Backpack For Boys Kpop BTS Bangtan ...,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/PALAY%C2%AE-Backpack-Ban...,4.1,37,"₹1,710","₹2,491",palay® bts backpack for boys kpop bts bangtan ...
9596,MOCA iPad Air 5th Generation Case 2022 / iPad ...,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/MOCA-Protective-Generati...,4.2,"1,133",₹475,"₹1,999",moca ipad air 5th generation case 2022 / ipad ...
9597,ZEBRONICS Zeb-Sound Bomb 5 TWS V5.0 Bluetooth ...,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Zebronics-Zeb-Sound-Blue...,3.5,"3,177",₹962,"₹3,999",zebronics zeb-sound bomb 5 tws v5.0 bluetooth ...
9598,LINQS® Original NXP Chip | Waterproof NFC Tag ...,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/LINQS-Sticker-Smartrac-C...,3.6,210,₹400,"₹1,000",linqs® original nxp chip | waterproof nfc tag ...


In [13]:
data['#keywords'] = data['#keywords'].str.replace('''[^\w\d\s]''','', regex=True)

In [14]:
stemmer = PorterStemmer()

In [15]:
def stemming(text):
    words = []
    for word in text.split(' '):
        words.append(stemmer.stem(word))
    return ' '.join(words)

In [16]:
data['#keywords'] = data['#keywords'].apply(stemming)

In [17]:
data['#keywords']

0         redmi 10 power power black 8gb ram 128gb storag
1       oneplu nord ce 2 lite 5g blue tide 6gb ram 128...
2       oneplu bullet z2 bluetooth wireless in ear ear...
3       samsung galaxi m33 5g mystiqu green 6gb 128gb ...
4       oneplu nord ce 2 lite 5g black dusk 6gb ram 12...
                              ...                        
9595    palay bt backpack for boy kpop bt bangtan scho...
9596    moca ipad air 5th gener case 2022  ipad air 4t...
9597    zebron zebsound bomb 5 tw v50 bluetooth truli ...
9598    linq origin nxp chip  waterproof nfc tag stick...
9599    pragati system medium grain 24x36 inch 10 oz p...
Name: #keywords, Length: 9600, dtype: object

In [18]:
cv = CountVectorizer(max_features = 5000, stop_words='english', dtype = np.uint8)

In [19]:
cv.fit(data['#keywords'])

CountVectorizer(dtype=<class 'numpy.uint8'>, max_features=5000,
                stop_words='english')

In [21]:
vector = cv.transform(data['#keywords']).toarray()

In [23]:
vector.shape

(9600, 5000)

In [24]:
similarity = cosine_similarity(vector)

In [25]:
del(vector)

In [27]:
data.drop(['#keywords'], axis=1,inplace=True)

In [29]:
similarity.shape

(9600, 9600)

In [30]:
similarity[0]

array([1.        , 0.28603878, 0.07106691, ..., 0.        , 0.        ,
       0.0836242 ])

In [31]:
def recommender(product):
    product_index =data[data['name']==product].index[0]
    similarity_list = list(enumerate(similarity[product_index]))
    similar_product =sorted(similarity_list, key=lambda x:x[1], reverse=True)[1:11]
    for idx, similar in similar_product:
        print(data.loc[idx]['name'])

In [35]:
data['name'].sample(10, random_state=5)

8414    ONUMTZ Newly Launched Smart Watch Charging Cable, Watch Holder Magnetic 24k Gold Plated 2 pin Cable, Compatible with T55, ...
2716    boAt Stone 620 Bluetooth Speaker with 12W RMS Stereo Sound, 10HRS Playtime, TWS Feature, IPX4, Multi-Compatibility Modes(G...
1360    VIHM 7 in 1 Electronic Cleaner kit, Cleaning Kit for Monitor Keyboard Airpods MacBook iPad iPhone iPod, Screen Dust Brush ...
1214    Apple Lightning to USB Camera Adapter, USB 3.0 OTG Cable for iPhone/iPad to Connect Card Reader, USB Flash Drive, U Disk, ...
420     boAt Rockerz 335 Bluetooth in Ear Neckband with Qualcomm aptX & CVC, Upto 30 Hours Playback, ASAP Charge, Signature Sound,...
9269    ENVOUS Newly Launched Fire Bolt Ring Smartwatch Charging Cable USB Fast Charger Magnetic Charging Cable Adapter (Charger o...
1994    DIGITEK (DTR-320 FT) Flexible Gorillapod Tripod for All Cameras and Mobile Phones with 360° Ball Head and Phone Holder, Ma...
5828            Lava A1 (Candy Blue), Number Talker, Smart AI 

In [37]:
product = "Lava A1 (Candy Blue), Number Talker, Smart AI Battery, 4 Days Battery Backup, Military Grade Certified, Keypad Mobile"
data[data['name']==product]

,name,image,link,ratings,no_of_ratings,discount_price,actual_price
5828,"Lava A1 (Candy Blue), Number Talker, Smart AI Battery, 4 Days Battery Backup, Military Grade Certified, Keypad Mobile",https://m.media-amazon.com/images/I/71llUMCdfZL._AC_UL320_.jpg,https://www.amazon.in/Lava-Notfication-recoding-Military-Certified/dp/B09F3J498L/ref=sr_1_5834?qid=1679134016&s=electronics&sr=1-5834,3.8,"3,384","₹1,087","₹1,299"


In [43]:
recommender("Lava Gem (Gold+Black), Speaker with Amplifier, PMMA 2.5D Glass, Military Grade Certified,1.3 MP Camera, Keypad Mobile")

Lava Gem (Blue Gold), Speaker with Amplifier, PMMA 2.5D Glass, Military Grade Certified,1.3 MP Camera, Keypad Mobile
Lava A1 2021(Blue Silver), Bluetooth Support, Smart AI Battery, Military Grade Certified,4 Days Battery Backup, Keypad Mobile
Lava A1 2021(Black Gold), Bluetooth Support, Smart AI Battery, Military Grade Certified,4 Days Battery Backup, Keypad Mobile
Lava A1 (Candy Blue), Number Talker, Smart AI Battery, 4 Days Battery Backup, Military Grade Certified, Keypad Mobile
Lava Hero Punch(Charcoal Grey),Stylish Design, 3 Day Battery Backup, FM with Recording, Keypad Mobile, Basic Mobile
MOSHKING Back Cover Case Compatible for iPhone 13 with Slide Camera || Heavy Duty Military Grade Protection Mobile Phone C...
MOSHKING Back Cover for Samsung Galaxy A73 5G with Slide Camera || Heavy Duty Military Grade Protection Mobile Phone Case ...
MOSHKING Back Cover for Samsung Galaxy A23 4G / 5G with Slide Camera | Heavy Duty Military Grade Protection Mobile Phone C...
Lava A5 (Gold), Mili

In [44]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))

In [45]:
pickle.dump(data,open('data.pkl','wb'))